# EDA Related to creating the Similarity Matrices


In [24]:
import os
import pandas as pd
import numpy as np
import re

In [5]:
data_dir = "../data/patientData"

In [26]:
# let's check some of the missing data
files = os.listdir(data_dir)
lab_files = [file for file in files if re.search("lab tests", file)]
lab_files

['lab tests_cross sect_antibodies.csv',
 'lab tests_cross sect_blood.csv',
 'lab tests_cross sect_breakdown products.csv',
 'lab tests_cross sect_enzymes.csv',
 'lab tests_cross sect_hepatitis.csv',
 'lab tests_cross sect_other lab.csv',
 'lab tests_cross sect_proteins.csv',
 'lab tests_time series.csv']

In [15]:
blood = pd.read_csv(data_dir + "/lab tests_cross sect_blood.csv")
print(len(blood.columns))
blood.head()

24


,SUBJECT_ID,min_fibrinogen,max_fibrinogen,avg_fibrinogen,min_platelet,max_platelet,avg_platelet,min_inr,max_inr,avg_inr,...,max_ptt,avg_ptt,first_ferritin,first_iron,first_iron_bind,first_transferrin,last_ferritin,last_iron,last_iron_bind,last_transferrin
0,4,NaN,NaN,NaN,201.0,388.0,286.428571,1.0,1.1,1.050000,...,33.2,32.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52,NaN,NaN,NaN,47.0,115.0,72.583333,1.7,2.7,2.241667,...,48.4,42.633333,640.0,90.0,185.0,142.0,NaN,NaN,NaN,NaN
2,78,NaN,NaN,NaN,45.0,53.0,47.750000,1.2,1.2,1.200000,...,39.0,38.350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117,79.0,253.0,158.25,24.0,98.0,52.565217,1.5,3.9,2.143750,...,150.0,56.434091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,140,NaN,NaN,NaN,65.0,86.0,72.666667,1.0,1.3,1.150000,...,31.5,30.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# let's check number of patients missing >50% of data
blood['total_count'] = blood.isnull().sum(axis=1)
round(blood.total_count.value_counts()/len(blood),4)*100

8     31.39
11    28.51
0     16.91
3     10.24
4      3.72
2      1.94
7      1.67
5      1.63
20     1.60
6      0.90
1      0.56
9      0.45
14     0.24
12     0.07
10     0.07
23     0.07
17     0.03
Name: total_count, dtype: float64

In [27]:
for file in lab_files:
    print(file)
    df = pd.read_csv(data_dir + "/" + file)
    
    print(len(df.columns))
    
    df['total_count'] = df.isnull().sum(axis=1)
    print(round(df.total_count.value_counts()/len(df),4)*100)
    
    print("\n ~~~~~~~~~~~~~~~~~~~~~~~~~~ \n")

lab tests_cross sect_antibodies.csv
5
0    100.0
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_blood.csv
24
8     31.39
11    28.51
0     16.91
3     10.24
4      3.72
2      1.94
7      1.67
5      1.63
20     1.60
6      0.90
1      0.56
9      0.45
14     0.24
12     0.07
10     0.07
23     0.07
17     0.03
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_breakdown products.csv
17
5     19.97
4     18.12
2     13.44
3     12.29
7     11.91
6     10.87
1      3.58
13     2.53
10     2.33
8      1.49
0      1.39
11     1.08
9      0.62
12     0.38
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_enzymes.csv
19
3     57.40
6     18.02
0     15.14
9      4.38
15     2.92
18     1.81
12     0.35
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_hepatitis.csv
5
0    100.0
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab t

Only one that is kind of bad is proteins. Otherwise, I can just remove patients from the study who have less than 50% of variables. 